In [1]:
import pandas as pd
import os
import numpy as np

In [14]:
#path_csv = "/home/gumougeot/all/data/3d_nucleus/all_manual/folds_x_sophie.csv"
path_csv = "C:\\Users\\mezquitap\\Documents\\nuclei_benchmark\\notebooks\\folds_x_sophie.csv"
path_img = "C:\\Users\\mezquitap\\Documents\\Dataset\\dataset\\cellpose\\cyto\\"
df = pd.read_csv(path_csv)
df.head()

,filename,fold_0,fold_x
0,KAKU4-wt--CRWN1-wt--CRWN4-wt_Cot_J13_STD_FIXE_...,0,0
1,KAKU4-wt--CRWN1-wt--CRWN4-wt_Cot_J13_STD_FIXE_...,2,2
2,KAKU4-wt--CRWN1-wt--CRWN4-wt_Cot_J13_STD_FIXE_...,2,11
3,KAKU4-wt--CRWN1-wt--CRWN4-wt_Cot_J13_STD_FIXE_...,2,2
4,KAKU4-wt--CRWN1-wt--CRWN4-wt_Cot_J13_STD_FIXE_...,0,0


### setters

In [7]:
def set_fold_0(df, test_ratio=0.2, val_ratio=0.2, seed=42):
    """
    define the test set and validation sets
    add it to the df in 'fold_0'
    {0: test, val: 1, train: 0}
    """
    np.random.seed(seed)
    l = np.array(df.iloc[:,0])
    
    # shuffle the list 
    permut = np.random.permutation(len(l))
    inv_permut = np.argsort(permut)
    # shuffled = l[permut] # shuffled contains the suffled list
    
    # split the shuffled list
    test_split = int(len(l)*test_ratio)
    val_split = int(len(l)*val_ratio)
    train_split = len(l)-test_split-val_split
    indices = np.array([0]*test_split+[1]*val_split+[2]*train_split)
    
    # unpermut the list of indice to get back the original
    sort_indices = indices[inv_permut]
    
    # add columns
    df['fold_0'] = sort_indices
    return df

df2 = set_fold_0(df)
# print(df2.head(25))

def set_fold_x(df, ratio=[0.5,0.25]):
    """
    create a new column with 4 types:
    {0: test, 1: val, 2->i: fold_1->(i-1)}
    TODO function: the validation set is retrieved from column "fold", fold number 0.
    params: 
     - nb_train_img: is the number of train images
    """
    # get the fold_0 list
    fold = np.array(df.fold_0)
    train_size = sum(fold==2)
    nb_imgs = [int(train_size*r) for r in ratio]
    
    # add the left overs
    nb_imgs += [1]*(train_size-sum(nb_imgs))
    print("number of images in each fold:", nb_imgs)
    
    # indices
    indices = []
    for i,r in enumerate(nb_imgs):
        indices += [i+2]*r
    
    # shuffle it 
    shuffle = np.random.permutation(indices)
    
    # replace the shuffle list in the original array where element==2
    j = 0
    for i in range(len(fold)):
        if fold[i]==2:
            fold[i]=shuffle[j]
            j+=1
    
    # put it back in the dataframe
    df['fold_x'] = fold
    return(df)


df3=set_fold_x(df=df2)
# df3.head(25)

number of images in each fold: [28, 14, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


### getters

In [19]:
def get_fold_x(df, fold=0):
    """
    Return the test set, the validation set and the training set
    """
    test_set = np.array(df[df['fold_x']==0].iloc[:,0])
    val_set = np.array(df[df['fold_x']==1].iloc[:,0])
    if fold==0: # return the full train set
        fold_set = np.array(df[df['fold_x']>1].iloc[:,0])
    else:
        fold_set = np.array(df[df['fold_x']==(fold+2)].iloc[:,0])
    return test_set, val_set, fold_set

(test_name, val_name, train_name) = get_fold_x(df, fold=0)


def name_to_path_in_array(array, path):
    new_array = []
    for name in array:
        new_array.append(path + name)
    return new_array
train = name_to_path_in_array(train_name, path_img)
val = name_to_path_in_array(val_name, path_img)
test = name_to_path_in_array(test_name, path_img)
print(train)
        


def get_nb_fold(df):
    """
    return the number of training folds.
    """
    return np.max(df['fold_x'])-2
print(get_nb_fold(df))

['C:\\Users\\mezquitap\\Documents\\Dataset\\dataset\\cellpose\\cyto\\KAKU4-wt--CRWN1-wt--CRWN4-wt_Cot_J13_STD_FIXE_H258_M3_1_C0.tif', 'C:\\Users\\mezquitap\\Documents\\Dataset\\dataset\\cellpose\\cyto\\KAKU4-wt--CRWN1-wt--CRWN4-wt_Cot_J13_STD_FIXE_H258_T3_13_C0.tif', 'C:\\Users\\mezquitap\\Documents\\Dataset\\dataset\\cellpose\\cyto\\KAKU4-wt--CRWN1-wt--CRWN4-wt_Cot_J13_STD_FIXE_H258_X2_21_C0.tif', 'C:\\Users\\mezquitap\\Documents\\Dataset\\dataset\\cellpose\\cyto\\KAKU4-wt--CRWN1-wt--CRWN4-wt_Cot_J13_STD_FIXE_H258_U1_1_C0.tif', 'C:\\Users\\mezquitap\\Documents\\Dataset\\dataset\\cellpose\\cyto\\KAKU4-wt--CRWN1-wt--CRWN4-wt_Cot_J13_STD_FIXE_H258_M3_9_C0.tif', 'C:\\Users\\mezquitap\\Documents\\Dataset\\dataset\\cellpose\\cyto\\KAKU4-wt--CRWN1-wt--CRWN4-wt_Cot_J13_STD_FIXE_H258_X2_12_C0.tif', 'C:\\Users\\mezquitap\\Documents\\Dataset\\dataset\\cellpose\\cyto\\KAKU4-wt--CRWN1-wt--CRWN4-wt_Cot_J13_STD_FIXE_H258_T1_28_C0.tif', 'C:\\Users\\mezquitap\\Documents\\Dataset\\dataset\\cellpose\\cy

In [20]:
import shutil

In [25]:
path_train = "C:\\Users\\mezquitap\\Documents\\Dataset\\dataset\\train\\"
path_test = "C:\\Users\\mezquitap\\Documents\\Dataset\\dataset\\test\\"
path_val = "C:\\Users\\mezquitap\\Documents\\Dataset\\dataset\\val\\"

def copy_files_from_dataset(array, array_name, new_path):
    for i in range(len(array)):
        shutil.copyfile(array[i], new_path + array_name[i])

copy_files_from_dataset(train, train_name, path_train)
copy_files_from_dataset(test, test_name, path_test)
copy_files_from_dataset(val, val_name, path_val)
